In [12]:
from multiprocessing import Process, Queue
import multiprocessing
from pathos.multiprocessing import Pool
from datasets.arrow_dataset import Dataset
from transformers import PreTrainedTokenizer
from typing import List, Dict
import torch
import random
import tqdm
import glob
from pathlib import Path
from datasets import concatenate_datasets, Dataset
from transformers import AutoTokenizer
from model import load_pretrained_bert_base

In [13]:
checkpoint = torch.load("../../models/bert_base_SST-2_93.58%/checkpoint_epoch001")

In [14]:
from transformers import AutoModelForSequenceClassification
from model import get_bert_config, BertForSequenceClassificationWithLoss
model = AutoModelForSequenceClassification.from_config(get_bert_config("base"))
torch.nn.DataParallel(BertForSequenceClassificationWithLoss(model)).load_state_dict(checkpoint["model_state_dict"])

<All keys matched successfully>

In [17]:
model.classifier.bias

Parameter containing:
tensor([ 0.0016,  0.0017, -0.0032, -0.0031, -0.0033], requires_grad=True)

In [11]:
torch.save(model, "../../models/bert_base_SST-2_93.58%/model.pt")

In [3]:
model = load_pretrained_bert_base()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [18]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
torch.save(model,"../../models/pretrained_bert.pt")

In [9]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenizer.decode([2000,2001,2002])

'to was he'

In [ ]:

mlm_dataset_path = Path("../../wikipedia_mlm/")

datasets = [
    Dataset.load_from_disk(path)
    for path in sorted(glob.glob(str(mlm_dataset_path / "*")))
]

In [1]:
import torch

torch.argmax(torch.tensor([0,0,0]))

/local/scratch-3/fwe21/miniconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor(2)

In [7]:
datasets

[Dataset({
     features: ['tokens', 'masked_tokens', 'is_masked'],
     num_rows: 162948
 }),
 Dataset({
     features: ['tokens', 'masked_tokens', 'is_masked'],
     num_rows: 172999
 }),
 Dataset({
     features: ['tokens', 'masked_tokens', 'is_masked'],
     num_rows: 160956
 }),
 Dataset({
     features: ['tokens', 'masked_tokens', 'is_masked'],
     num_rows: 175229
 }),
 Dataset({
     features: ['tokens', 'masked_tokens', 'is_masked'],
     num_rows: 162693
 }),
 Dataset({
     features: ['tokens', 'masked_tokens', 'is_masked'],
     num_rows: 168274
 }),
 Dataset({
     features: ['tokens', 'masked_tokens', 'is_masked'],
     num_rows: 167903
 }),
 Dataset({
     features: ['tokens', 'masked_tokens', 'is_masked'],
     num_rows: 161565
 }),
 Dataset({
     features: ['tokens', 'masked_tokens', 'is_masked'],
     num_rows: 162859
 }),
 Dataset({
     features: ['tokens', 'masked_tokens', 'is_masked'],
     num_rows: 165703
 }),
 Dataset({
     features: ['tokens', 'masked_token

In [9]:
import os
"\n".join(os.popen('free -t -m').readlines())

['              total        used        free      shared  buff/cache   available\n',
 'Mem:         128800       41259       33827           5       53713       86274\n',
 'Swap:          1766        1520         245\n',
 'Total:       130566       42780       34072\n']

In [35]:
import os, psutil
process = psutil.Process(os.getpid())
print(process.memory_info().rss//1000000)

1689


In [13]:
dataset = concatenate_datasets(datasets)

Transform <function _concatenate_map_style_datasets at 0x7f33d57a59d0> couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [21]:
dataset.set_format("torch")

In [31]:
x = 0
for i in tqdm.tqdm(range(1000)):
    x += len(dataset[random.randint(0,len(dataset)-1)])

100%|██████████| 1000/1000 [00:11<00:00, 84.48it/s]


3000